### Importing all the required libraries

In [3]:
import pandas as pd
#!pip install sqlalchemy
import sqlalchemy as sc
from sqlalchemy import create_engine, text

In [7]:
df = pd.read_csv('//Users//sayantrinisaha//Desktop//Google LLM VertexAi//UNZIP-ME-FOR-NOTEBOOKS//05-Lab-NLP-Database-Assistant//penguins.csv')

In [9]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [19]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [21]:
data = df.to_sql(name='Penguins', con=temp_db)

2024-06-20 16:05:17,273 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-20 16:05:17,283 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2024-06-20 16:05:17,284 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-20 16:05:17,287 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2024-06-20 16:05:17,288 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-06-20 16:05:17,291 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2024-06-20 16:05:17,294 INFO sqlalchemy.engine.Engine [no key 0.00270s] ()
2024-06-20 16:05:17,295 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2024-06-20 16:05:17,296 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2024-06-20 16:05:17,405 INFO sqlalchemy.engine.Engine INSERT INTO "Penguins" ("index", species, island, bill_length_m

In [23]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * FROM Penguins"))

2024-06-20 16:06:37,568 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-20 16:06:37,575 INFO sqlalchemy.engine.Engine SELECT * FROM Penguins
2024-06-20 16:06:37,577 INFO sqlalchemy.engine.Engine [generated in 0.00858s] ()
2024-06-20 16:06:37,579 INFO sqlalchemy.engine.Engine ROLLBACK


In [25]:
result.all()

[(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
 (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE'),
 (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE'),
 (3, 'Adelie', 'Torgersen', None, None, None, None, None),
 (4, 'Adelie', 'Torgersen', 36.7, 19.3, 193.0, 3450.0, 'FEMALE'),
 (5, 'Adelie', 'Torgersen', 39.3, 20.6, 190.0, 3650.0, 'MALE'),
 (6, 'Adelie', 'Torgersen', 38.9, 17.8, 181.0, 3625.0, 'FEMALE'),
 (7, 'Adelie', 'Torgersen', 39.2, 19.6, 195.0, 4675.0, 'MALE'),
 (8, 'Adelie', 'Torgersen', 34.1, 18.1, 193.0, 3475.0, None),
 (9, 'Adelie', 'Torgersen', 42.0, 20.2, 190.0, 4250.0, None),
 (10, 'Adelie', 'Torgersen', 37.8, 17.1, 186.0, 3300.0, None),
 (11, 'Adelie', 'Torgersen', 37.8, 17.3, 180.0, 3700.0, None),
 (12, 'Adelie', 'Torgersen', 41.1, 17.6, 182.0, 3200.0, 'FEMALE'),
 (13, 'Adelie', 'Torgersen', 38.6, 21.2, 191.0, 3800.0, 'MALE'),
 (14, 'Adelie', 'Torgersen', 34.6, 21.1, 198.0, 4400.0, 'MALE'),
 (15, 'Adelie', 'Torgersen', 36.6, 17.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [29]:
def create_prefix(query):
    prefix = f""" Return a SQL statement that answers the following query:
    {query}

    For a table called 'Penguins' with the following properties:

     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object

     here are some example rows:
     (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE')
     (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
     (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE')

     Only return the SQL statement for the query.
    """
    return prefix

In [31]:
def user_input():
    query = input("Ask a question about the Penguins table")
    return create_prefix(query)

In [33]:
prefix = user_input()

Ask a question about the Penguins table How many penguins are there in the table?


In [35]:
print(prefix)

 Return a SQL statement that answers the following query:
    How many penguins are there in the table?

    For a table called 'Penguins' with the following properties:

     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object

     here are some example rows:
     (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE')
     (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
     (2, 'Adelie', 'Torgersen', 40.3, 18.0, 195.0, 3250.0, 'FEMALE')

     Only return the SQL statement for the query.
    


In [41]:
import os

In [43]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '//Users//sayantrinisaha//Desktop//psyched-metrics-426921-h0-4a9007b858ad.json'

In [45]:
from google.cloud import aiplatform

In [47]:
import vertexai

In [49]:
from vertexai.language_models import CodeGenerationModel

In [51]:
codegen_model = CodeGenerationModel.from_pretrained('code-bison')

In [53]:
print("Hello, I am your AI database assistant")
print('\n\n')
prefix = user_input()
result = codegen_model.predict(prefix=prefix)

Hello, I am your AI database assistant





Ask a question about the Penguins table How many Penguins are female?


In [61]:
result

```sql
SELECT COUNT(*)
FROM Penguins
WHERE sex = 'FEMALE';
```

In [63]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql','').replace('```','').replace('\n',' ');
clean_sql(result.text)

" SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE'; "

In [83]:
def nlp_sql_assistant():
    print("Hello, I am your AI database assistant")
    print('\n\n')
    prefix = user_input()
    result = codegen_model.predict(prefix=prefix)
    sql_comm = clean_sql(result.text)
    with temp_db.connect() as conn:
        result_final = conn.execute(text(sql_comm))
    print('\n\n\n=================================')
    print("here are your results:")
    print(result_final.all())

In [85]:
nlp_sql_assistant()

Hello, I am your AI database assistant





Ask a question about the Penguins table How many Penguins are female?


2024-06-20 17:31:04,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-20 17:31:04,161 INFO sqlalchemy.engine.Engine  SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE'; 
2024-06-20 17:31:04,163 INFO sqlalchemy.engine.Engine [cached since 136.2s ago] ()
2024-06-20 17:31:04,165 INFO sqlalchemy.engine.Engine ROLLBACK



here are your results:
[(165,)]
